In [ ]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objs as go
datafile = 'wine-reviews\winemag-data_first150k.csv'
#数据导入
wdata =pd.read_csv(datafile,encoding='utf-8')
wdata.info()
#将每个属性的描述性统计量展示
explore=wdata.describe(percentiles=[],include='all').T
#计算属性对应的空值个数
print('country的缺失值个数为:',wdata['country'].isnull().sum())
print('description的缺失值个数为:',wdata['description'].isnull().sum())
print('designation的缺失值个数为:',wdata['designation'].isnull().sum())
print('points的缺失值个数为:',wdata['points'].isnull().sum())
print('price的缺失值个数为:',wdata['price'].isnull().sum())
print('province的缺失值个数为:',wdata['province'].isnull().sum())
print('region_1的缺失值个数为:',wdata['region_1'].isnull().sum())
print('region_2的缺失值个数为:',wdata['region_2'].isnull().sum())
print('variety的缺失值个数为:',wdata['variety'].isnull().sum())
print('winery的缺失值个数为:',wdata['winery'].isnull().sum())
#求price属性的五数概括
import numpy as np
price=wdata['price']
newprice = price.dropna(axis=0)
newprice = newprice.sort_values()
price_d = newprice.values
print(price_d)
min_price = min(price_d) #找出数组中的最小值
max_price = max(price_d) #找出数组中的最大值
Q1=np.percentile(price_d,25)
Median=np.median(price_d)
Q3=np.percentile(price_d,75)
print(min_price,Q1,Median,Q3,max_price)
wdata = [
go.Box(
y=[min_price, Q1, Median, Q3, max_price] 
)
]
py.plot(wdata) # 绘图
# 绘制直方图
wdata0=wdata
any(wdata0.price.isnull())
wdata0.dropna(subset=['price'], inplace=True)
plt.hist(x = wdata0.price, # 指定绘图数据
          bins = 100, # 指定直方图中条块的个数
          color = 'steelblue', # 指定直方图的填充色
          edgecolor = 'black' # 指定直方图的边框色
          )
# 添加x轴和y轴标签
plt.xlabel('price')
plt.ylabel('frequency')
#plt.title('')
# 显示图形
plt.show()
swdata=wdata.sample(n=2000,axis=0)
#直接删除
swdata1 = swdata.dropna(axis=0, subset=['price'])
f = go.Figure(go.Scatter(x=swdata1.points, y=swdata1.price, text=swdata1.designation, mode='markers', name='2007'))
f.update_xaxes(title_text='points', type='log')
f.update_yaxes(title_text='price')
py.plot(f, filename='pandas-multiple-scatter')

#用均值替代缺失值
sprice=swdata['price']
sprice = sprice.dropna(axis=0)
sprice_d = sprice.values
wMean=np.mean(sprice_d)
sprice=sprice.replace(np.nan,wMean)
swdata2=swdata
swdata2['price']=sprice
fm = go.Figure(go.Scatter(x=swdata2.points, y=swdata2.price, text=swdata2.designation, mode='markers', name='2007'))
fm.update_xaxes(title_text='points', type='log')
fm.update_yaxes(title_text='price')
py.plot(fm, filename='pandas-multiple-scatter')
#利用KNN填补缺失值
from fancyimpute import KNN
sprice=swdata['price']
sprice_d = sprice.values
fill_knn = KNN(k=3).fit_transform(swdata)
data = pd.DataFrame(fill_knn)
print(data.head())
#随机森林填充缺失值
from sklearn.ensemble import RandomForestRegressor
# 分成已知该特征和未知该特征两部分
known = swdata[swdata.c.notnull()].as_matrix()
uknown = swdata[swdata.c.isnull()].as_matrix()
# X为特征属性值
X = known[:, 1:3]
# print(X[0:10])
# Y为结果标签
y = known[:, 0]
print(y)
# 训练模型
rf = RandomForestRegressor(random_state=0, n_estimators=200, max_depth=3, n_jobs=-1)
rf.fit(X, y)
# 预测缺失值
predicted = rf.predict(uknown[:, 1:3])
print(predicted)
#将预测值填补原缺失值
sdata.loc[(sdata.c.isnull()), 'c'] = predicted